In [7]:
import pandas as pd
df = pd.read_parquet("C:/Users/andya/Downloads/dffeatures.parquet")
print(df.head())

KeyboardInterrupt: 

In [ ]:
df.head()

,volume,vwap,open,close,high,low,timestamp,transactions,datetime,ticker,minute,hour,dayofweek,is_open_hour,hl_spread,oc_return,log_return,close_lag1,close_roll_mean_5,volume_roll_std_15
0,17632.0,137.0174,137.420,137.2600,137.565,136.85,1680528600000,105,2023-04-03 13:30:00,A,30,13,0,True,0.715,-0.001164,NaN,NaN,NaN,NaN
1,1011.0,137.2530,137.255,137.3200,137.320,137.25,1680528660000,28,2023-04-03 13:31:00,A,31,13,0,True,0.070,0.000474,0.000437,137.26,NaN,NaN
2,697.0,137.2364,137.220,137.3300,137.330,136.95,1680528720000,18,2023-04-03 13:32:00,A,32,13,0,True,0.380,0.000802,0.000073,137.32,NaN,NaN
3,4004.0,137.3055,136.990,137.5700,137.570,136.99,1680528780000,79,2023-04-03 13:33:00,A,33,13,0,True,0.580,0.004234,0.001746,137.33,NaN,NaN
4,7244.0,137.4399,137.690,137.3784,137.720,137.11,1680528840000,107,2023-04-03 13:34:00,A,34,13,0,True,0.610,-0.002263,-0.001394,137.57,137.37168,NaN


In [ ]:
pip install pyspark findspark


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.types import (
    StructType, StructField,
    StringType, DoubleType, BooleanType
)

# 1. 建立 SparkSession
spark = SparkSession.builder \
    .appName("FeatureAnalysis") \
    .getOrCreate()

# 2. 定义“精简”schema，只包含你关心且类型正确的列
schema = StructType([
    StructField("ticker",        StringType(),  True),
    StructField("hl_spread",     DoubleType(),  True),
    StructField("oc_return",     DoubleType(),  True),
    StructField("volume",        DoubleType(),  True),  # 改成 DoubleType
    StructField("is_open_hour",  BooleanType(), True),
])

# 3. 读 parquet 时带上 schema，Spark 会跳过其余所有列的推断
df = (
    spark.read
         .schema(schema)
         .parquet("file:///C:/Users/andya/Downloads/dffeatures.parquet")
)

# 4. 建临时视图，用 SQL 分析
df.createOrReplaceTempView("features")

# 5. 执行查询
result = spark.sql("""
    SELECT
      ticker,
      AVG(hl_spread)     AS avg_hl_spread,
      AVG(oc_return)     AS avg_return,
      stddev_pop(volume) AS vol_stddev
    FROM features
    WHERE is_open_hour = TRUE
    GROUP BY ticker
    ORDER BY avg_return DESC
    LIMIT 10
""")

# 6. 显示结果
result.show(truncate=False)

# 7. 如不再需要，可关闭 Spark
# spark.stop()


+------+--------------------+---------------------+------------------+
|ticker|avg_hl_spread       |avg_return           |vol_stddev        |
+------+--------------------+---------------------+------------------+
|ALNY  |0.22794516604510554 |1.8248372561491283E-5|5008.22528894602  |
|FLUT  |0.1977968052437489  |1.7291626995166514E-5|13015.219605447162|
|CVNA  |0.255732174119451   |1.6904387971735355E-5|51872.933769178206|
|FICO  |0.7712305359327496  |1.5471581641603663E-5|1001.8876694490925|
|TW    |0.0759049954027869  |1.4769943953664653E-5|3905.101866880599 |
|EQIX  |0.49408498746661966 |1.341561522198459E-5 |2511.270436887145 |
|AMX   |0.011745936270765185|1.1426383527480078E-5|13687.872264852642|
|HUBS  |0.605693855711273   |1.1075725171105891E-5|2795.7816182160354|
|HEI   |0.11672584200873673 |9.860211290031245E-6 |4068.6606716436136|
|CRH   |0.05638075897011938 |7.545789309962562E-6 |16645.359350873685|
+------+--------------------+---------------------+------------------+



In [4]:
pip install duckdb

Note: you may need to restart the kernel to use updated packages.


   ---------------------------------------- 0.0/11.4 MB ? eta -:--:--
   --- ------------------------------------ 1.0/11.4 MB 10.1 MB/s eta 0:00:02
   ---------- ----------------------------- 2.9/11.4 MB 8.8 MB/s eta 0:00:01
   --------------- ------------------------ 4.5/11.4 MB 8.6 MB/s eta 0:00:01
   ----------------------- ---------------- 6.6/11.4 MB 8.8 MB/s eta 0:00:01
   ----------------------------- ---------- 8.4/11.4 MB 8.7 MB/s eta 0:00:01
   ------------------------------------- -- 10.7/11.4 MB 9.1 MB/s eta 0:00:01
   ---------------------------------------- 11.4/11.4 MB 9.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import duckdb

# 1. read csv and then parse datetime
df = pd.read_csv(
    "C:/Users/andya/Downloads/merged_raw.csv",
    parse_dates=["datetime"],
    infer_datetime_format=True
)

# 2. rename every column
df = df.rename(columns={
    "v":  "volume",
    "vw": "vwap",
    "o":  "open",
    "c":  "close",
    "h":  "high",
    "l":  "low",
    "t":  "timestamp",
    "n":  "transactions"
})

# 3. clean missing value
price_cols = ["open", "high", "low", "close"]
df[price_cols] = df.groupby("ticker")[price_cols].ffill()

df["volume"] = df["volume"].fillna(0)

# To fix the error, I use 'transform'to keep index
df["vwap"] = df.groupby("ticker")["vwap"] \
              .transform(lambda x: x.interpolate())

#  deal with time column
df["minute"]      = df["datetime"].dt.minute
df["hour"]        = df["datetime"].dt.hour
df["dayofweek"]   = df["datetime"].dt.dayofweek
df["is_open_hour"] = df["hour"].between(9, 16)

# 5. Use DuckDB to feature
con = duckdb.connect(database=":memory:")
con.register("raw", df)

q = """
SELECT
  *,
  high - low                              AS hl_spread,
  (close - open) / open                   AS oc_return,
  LOG(close / LAG(close) OVER win1)       AS log_return,
  LAG(close) OVER win1                    AS close_lag1,
  AVG(close) OVER win5                    AS close_roll_mean_5,
  STDDEV_POP(volume) OVER win15           AS volume_roll_std_15
FROM raw
WINDOW
  win1 AS (PARTITION BY ticker ORDER BY datetime ROWS BETWEEN 1 PRECEDING AND CURRENT ROW),
  win5 AS (PARTITION BY ticker ORDER BY datetime ROWS BETWEEN 4 PRECEDING AND CURRENT ROW),
  win15 AS (PARTITION BY ticker ORDER BY datetime ROWS BETWEEN 14 PRECEDING AND CURRENT ROW)
ORDER BY ticker, datetime
"""
df_features = con.execute(q).df()

# 6. get Parquet
df_features.to_parquet(
    "C:/Users/andya/Downloads/dffeatures_duckdb.parquet",
    index=False
)

#run 19 min


C:\Users\andya\AppData\Local\Temp\ipykernel_3636\3239482965.py:6: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv(


In [1]:
import pandas as pd
df = pd.read_parquet("C:/Users/andya/Downloads/dffeatures_duckdb.parquet")
df.head()

,volume,vwap,open,close,high,low,timestamp,transactions,datetime,ticker,minute,hour,dayofweek,is_open_hour,hl_spread,oc_return,log_return,close_lag1,close_roll_mean_5,volume_roll_std_15
0,17632.0,137.0174,137.420,137.2600,137.565,136.85,1680528600000,105,2023-04-03 13:30:00,A,30,13,0,True,0.715,-0.001164,NaN,NaN,137.260000,0.000000
1,1011.0,137.2530,137.255,137.3200,137.320,137.25,1680528660000,28,2023-04-03 13:31:00,A,31,13,0,True,0.070,0.000474,0.000190,137.26,137.290000,8310.500000
2,697.0,137.2364,137.220,137.3300,137.330,136.95,1680528720000,18,2023-04-03 13:32:00,A,32,13,0,True,0.380,0.000802,0.000032,137.32,137.303333,7910.263811
3,4004.0,137.3055,136.990,137.5700,137.570,136.99,1680528780000,79,2023-04-03 13:33:00,A,33,13,0,True,0.580,0.004234,0.000758,137.33,137.370000,6931.662607
4,7244.0,137.4399,137.690,137.3784,137.720,137.11,1680528840000,107,2023-04-03 13:34:00,A,34,13,0,True,0.610,-0.002263,-0.000605,137.57,137.371680,6225.395685
